# Data Base

Florian EPAIN
20105403, MASTER IL group 3

## Belatedly Installation

We can visualize blank `.csv` file using `vscode` and the extension `Rainbow CSV`.

### Docs

- I used the [Psycopg2 tutorials](https://www.psycopg.org/docs/install.html#quick-install) to connect my notebook to my database.
- [SQL Datatypes](https://www.w3schools.com/sql/sql_datatypes.asp)

In [ ]:
!pip install -U pip

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.6 MB/s eta 0:00:00m eta 0:00:010:01


In [ ]:
!pip install duckdb==0.9.1

In [3]:
# If the Server postgres is down
import duckdb
cursor = duckdb.connect()
# print(cursor.execute('SELECT 42').fetchall())

In [1]:
import psycopg2
from getpass import getpass
  
# Connection establishment 
conn = psycopg2.connect( 
    database="fepain", 
    user='fepain', 
    password=getpass(), 
    host='chausey.istic.univ-rennes1.fr', 
    port='5432'
)
  
conn.autocommit = True
  
# Creating a cursor object 
cursor = conn.cursor()

········


OperationalError: connection to server at "chausey.istic.univ-rennes1.fr" (148.60.15.141), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [5]:
# Query to create a table 
sql = ''' CREATE TABLE TripDataSample 
( 
    medallion VARCHAR(32), 
    hack_licence VARCHAR(32), 
    vendor_id VARCHAR(10), 
    rate_code INT,
    store_and_fwd_flag VARCHAR(10),
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    passenger_count INT,
    trip_time_in_secs INT,
    trip_distance FLOAT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT
); '''
  
# --- Executing above query ---
cursor.execute(sql) # duckdb & postgres
print("Table has been created successfully D:")

Table has been created successfully D:


---

We now import the `csv` file, using the [Cursor Copy - tutorials](https://www.psycopg.org/docs/cursor.html#cursor.copy_from) and the [COPY docs](https://www.postgresql.org/docs/current/sql-copy.html). I can't use `copy_from()`, I will use `copy_expert()` as mentionned in this [github issue](https://github.com/psycopg/psycopg2/issues/1294)

---

In [6]:
# Display the table 
cursor.execute('SELECT * FROM TripDataSample')
print(cursor.fetchall())

[]


In [9]:
# First Try
sql = ''' COPY TripDataSample FROM 'assets/data/trip_data-sample.csv' With CSV HEADER DELIMITER as ','; '''
cursor.execute(sql)
print(cursor.fetchall())

[(9,)]


In [11]:
# Import CSV in PostgreSQL
csv_file_path = "assets/data/trip_data-sample.csv"
# cursor.copy_from(f, 'TripDataSample')

with open(csv_file_path, 'r') as f:
    cursor.copy_expert(
    "COPY TripDataSample FROM STDIN With CSV HEADER DELIMITER as ','",
    f)

# Display the table 
cursor.execute('SELECT * FROM TripDataSample')
print(cursor.fetchall())

AttributeError: 'duckdb.duckdb.DuckDBPyConnection' object has no attribute 'copy_expert'

In [22]:
from sys import stdout

cursor.copy_expert("COPY TripDataSample TO STDOUT WITH CSV HEADER", stdout)

medallion,hack_licence,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude


In [10]:
# Import CSV in DuckDB
cursor.execute('SELECT * FROM \'trip_data-sample.csv\';')
print(cursor.fetchall())

[('89D227B655E5C82AECF13C3F540D4CF4', 'BA96DE419E711691B9445D6A6307C170', 'CMT', 1, 'N', datetime.datetime(2013, 1, 1, 15, 11, 48), datetime.datetime(2013, 1, 1, 15, 18, 10), 4, 382, 1.0, -73.978165, 40.757977, -73.989838, 40.751171), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 6, 0, 18, 35), datetime.datetime(2013, 1, 6, 0, 22, 54), 1, 259, 1.5, -74.006683, 40.731781, -73.994499, 40.75066), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 5, 18, 49, 41), datetime.datetime(2013, 1, 5, 18, 54, 23), 1, 282, 1.1, -74.004707, 40.73777, -74.009834, 40.726002), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', '51EE87E3205C985EF8431D850C786310', 'CMT', 1, 'N', datetime.datetime(2013, 1, 7, 23, 54, 15), datetime.datetime(2013, 1, 7, 23, 58, 20), 2, 244, 0.7, -73.974602, 40.759945, -73.984734, 40.759388), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', '51EE87E3205C985EF8431D850C786

In [ ]:
# Import Big CSV in DuckDB
cursor.execute('SELECT * FROM \'trip_data_1.csv\';')
print(cursor.fetchall())

# Analyze Queries


- [EXPLAIN docs](https://www.postgresql.org/docs/current/sql-explain.html)
- <https://explain.dalibo.com/>

In [11]:
# Query 0
query = 'SELECT * FROM TripDataSample WHERE medallion = \'0BD7C8F5BA12B88E0B67BED28BEA73D8\';'
cursor.execute(query)
print(cursor.fetchall())

print("-------------")

explain = 'EXPLAIN ' + query
cursor.execute(explain)
print(cursor.fetchall())

[('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 6, 0, 18, 35), datetime.datetime(2013, 1, 6, 0, 22, 54), 1, 259, 1.5, -74.00668334960938, 40.731781005859375, -73.99449920654297, 40.75065994262695), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 5, 18, 49, 41), datetime.datetime(2013, 1, 5, 18, 54, 23), 1, 282, 1.100000023841858, -74.00470733642578, 40.737770080566406, -74.00983428955078, 40.72600173950195), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 6, 0, 18, 35), datetime.datetime(2013, 1, 6, 0, 22, 54), 1, 259, 1.5, -74.00668334960938, 40.731781005859375, -73.99449920654297, 40.75065994262695), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 5, 18, 49, 41), datetime.datetime(2013, 1, 5, 18, 54, 23), 1, 282, 1.1000

In [4]:
# Query 1
query = 'SELECT * FROM TripDataSample WHERE passenger_count >= 2;'
cursor.execute(query)
print(cursor.fetchall())

print("-------------")

explain = 'EXPLAIN ' + query
cursor.execute(explain)
print(cursor.fetchall())

[('89D227B655E5C82AECF13C3F540D4CF4', 'BA96DE419E711691B9445D6A6307C170', 'CMT', 1, 'N', datetime.datetime(2013, 1, 1, 15, 11, 48), datetime.datetime(2013, 1, 1, 15, 18, 10), 4, 382, 1.0, -73.978165, 40.757977, -73.989838, 40.751171), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', '51EE87E3205C985EF8431D850C786310', 'CMT', 1, 'N', datetime.datetime(2013, 1, 7, 23, 54, 15), datetime.datetime(2013, 1, 7, 23, 58, 20), 2, 244, 0.7, -73.974602, 40.759945, -73.984734, 40.759388), ('0B57B9633A2FECD3D3B1944AFC7471CF', 'CCD4367B417ED6634D986F573A552A62', 'CMT', 1, 'N', datetime.datetime(2013, 1, 7, 12, 39, 18), datetime.datetime(2013, 1, 7, 13, 10, 56), 3, 1898, 10.7, -73.989937, 40.756775, -73.86525, 40.77063)]
-------------
[('Seq Scan on tripdatasample  (cost=0.00..13.00 rows=80 width=308)',), ('  Filter: (passenger_count >= 2)',)]


In [5]:
# Query 2
query = 'SELECT * FROM TripDataSample WHERE passenger_count = 1;'
cursor.execute(query)
print(cursor.fetchall())

print("-------------")

explain = 'EXPLAIN ' + query
cursor.execute(explain)
print(cursor.fetchall())

[('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 6, 0, 18, 35), datetime.datetime(2013, 1, 6, 0, 22, 54), 1, 259, 1.5, -74.006683, 40.731781, -73.994499, 40.75066), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 5, 18, 49, 41), datetime.datetime(2013, 1, 5, 18, 54, 23), 1, 282, 1.1, -74.004707, 40.73777, -74.009834, 40.726002), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', '51EE87E3205C985EF8431D850C786310', 'CMT', 1, 'N', datetime.datetime(2013, 1, 7, 23, 25, 3), datetime.datetime(2013, 1, 7, 23, 34, 24), 1, 560, 2.1, -73.97625, 40.748528, -74.002586, 40.747868), ('20D9ECB2CA0767CF7A01564DF2844A3E', '598CCE5B9C1918568DEE71F43CF26CD2', 'CMT', 1, 'N', datetime.datetime(2013, 1, 7, 15, 27, 48), datetime.datetime(2013, 1, 7, 15, 38, 37), 1, 648, 1.7, -73.966743, 40.764252, -73.983322, 40.743763), ('496644932DF3932605C22C7926FF0FE0', '513189AD756FF14FE670D10B92FAF04

In [6]:
# Aggregate 1
aggregate = 'SELECT medallion, AVG(trip_distance) FROM TripDataSample WHERE passenger_count = 1 GROUP BY medallion;'
cursor.execute(aggregate)
print(cursor.fetchall())

print("-------------")

explain = 'EXPLAIN ' + aggregate
cursor.execute(explain)
print(cursor.fetchall())

[('0BD7C8F5BA12B88E0B67BED28BEA73D8', 1.3), ('20D9ECB2CA0767CF7A01564DF2844A3E', 1.7), ('2C0E91FF20A856C891483ED63589F982', 0.8), ('496644932DF3932605C22C7926FF0FE0', 0.8), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', 2.1)]
-------------
[('GroupAggregate  (cost=13.01..13.03 rows=1 width=90)',), ('  Group Key: medallion',), ('  ->  Sort  (cost=13.01..13.02 rows=1 width=90)',), ('        Sort Key: medallion',), ('        ->  Seq Scan on tripdatasample  (cost=0.00..13.00 rows=1 width=90)',), ('              Filter: (passenger_count = 1)',)]


In [8]:
# Aggregate 2
aggregate = 'SELECT AVG(passenger_count) FROM TripDataSample GROUP BY pickup_datetime;'
cursor.execute(aggregate)
print(cursor.fetchall())

print("-------------")

explain = 'EXPLAIN ' + aggregate
cursor.execute(explain)
print(cursor.fetchall())

[(Decimal('4.0000000000000000'),), (Decimal('2.0000000000000000'),), (Decimal('1.00000000000000000000'),), (Decimal('1.00000000000000000000'),), (Decimal('1.00000000000000000000'),), (Decimal('1.00000000000000000000'),), (Decimal('3.0000000000000000'),), (Decimal('1.00000000000000000000'),), (Decimal('1.00000000000000000000'),)]
-------------
[('HashAggregate  (cost=13.60..16.10 rows=200 width=40)',), ('  Group Key: pickup_datetime',), ('  ->  Seq Scan on tripdatasample  (cost=0.00..12.40 rows=240 width=12)',)]


In [26]:
# Aggregate 3
query = 'SELECT medallion, AVG(passenger_count) as avg_pass_count, AVG(trip_distance) as avg_trip_dist FROM TripDataSample GROUP BY medallion'
cursor.execute(query)
print('(medallion, avg_passenger_count, avg_trip_dist):')
print(cursor.fetchall())

print("-------------")

global_avg = 'SELECT AVG(trip_distance) FROM TripDataSample'
cursor.execute(global_avg)
print('global avg trip distance:')
print(cursor.fetchall())

print("-------------")

aggregate = 'SELECT avg_pass_count FROM (' + query + ') WHERE avg_trip_dist > (' + global_avg + ');'
cursor.execute(aggregate)
print('A3:')
print(cursor.fetchall())

print("-------------")

explain = 'EXPLAIN ' + aggregate
cursor.execute(explain)
print(cursor.fetchall())

(medallion, avg_passenger_count, avg_trip_dist):
[('2C0E91FF20A856C891483ED63589F982', 1.0, 0.800000011920929), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', 1.0, 1.300000011920929), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', 1.5, 1.3999999463558197), ('89D227B655E5C82AECF13C3F540D4CF4', 4.0, 1.0), ('0B57B9633A2FECD3D3B1944AFC7471CF', 3.0, 10.699999809265137), ('496644932DF3932605C22C7926FF0FE0', 1.0, 0.800000011920929), ('20D9ECB2CA0767CF7A01564DF2844A3E', 1.0, 1.7000000476837158)]
-------------
global avg trip distance:
[(2.2666666441493564,)]
-------------
A3:
[(3.0,)]
-------------
[('physical_plan', '┌───────────────────────────┐                             \n│         PROJECTION        │                             \n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             \n│       avg_pass_count      │                             \n└─────────────┬─────────────┘                                                          \n┌─────────────┴─────────────┐                             \n│      NESTED

## Compare DuckDB and PostgreSQL

- OLAP vs OLTP

In [13]:
# Closing the connection 
conn.close()